# Discrete optimization: relaxation

## Introduction to optimization and operations research

Michel Bierlaire


In [ ]:

import itertools
import sys
import warnings

import numpy as np
from teaching_optimization.simplex_tableau import SimplexAlgorithmTableau
from teaching_optimization.tableau import SimplexTableau


In this lab, you will compare an **integer** model with its **linear relaxation** on a small
investment decision. You will formulate the binary model, relax it to a linear
optimization problem, solve the relaxation with the simplex tableau, and interpret the
fractional solution as a **bound** (not a decision). Then you will perform **full enumeration**
of the integer solutions to find the true optimum and compare it to the relaxation’s bound.
The goal is to understand why relaxations are useful (fast, provide bounds and insight) but
sometimes **misleading** for decisions, and how combining bounds with exact search leads to
reliable recommendations.

The company Alpiq is investing in hydro-electricity. The engineers
have identified four potential location to build new dams. For each
location, they have assessed the investment costs as well as the
long-term expected revenues:

|               |   Location     | Cost (mCHF)    | Revenues (mCHF)|
:--------------:|:--------------:|:--------------:|:--------------:|
|      1        |    China       |      50        |       160      |
|      2        |     Iran       |      70        |       220      |
|      3        |    Brazil      |      40        |       120      |
|      4        |    India       |      30        |        80      |

The company has a budget of 140 mCHF. Where should the company invest
in new dams?

## Question 1
What type of optimization problem is it?

This is a knapsack problem.

## Question 2
Formulate the problem as an integer optimization problem.

The decision variables are binary variables: $x_i$ is 1 if a dam
is built at location $i$, and 0 otherwise.

The objective is to  maximize the expected revenues:
$$
\max_{x\in\{0,1\}^4} 16 x_1 + 22 x_2 + 12 x_3 + 8 x_4.
$$

Note that we are modeling the problem using 10mCHF as the base unit.

The budget constraint is given by
$$
5 x_1 + 7 x_2 + 4 x_3 + 3 x_4 \leq 14.
$$

Therefore, the integer optimization problem is:
$$
\max_{x\in\{0,1\}^4} 16 x_1 + 22 x_2 + 12 x_3 + 8 x_4,
$$
subject to
\begin{align*}
5 x_1 + 7 x_2 + 4 x_3 + 3 x_4 &\leq 14, \\
x_1, x_2, x_3, x_4 & \in \{ 0,1 \}.
\end{align*}

## Question 3
Solve the relaxation of the problem, and use the
solution of the relaxation to advise the company.

We first write the relaxation.
$$
\max_{x\in\mathbb{R}^4} 16 x_1 + 22 x_2 + 12 x_3 + 8 x_4,
$$
subject to
\begin{align*}
5 x_1 + 7 x_2 + 4 x_3 + 3 x_4 &\leq 14, \\
x_1, x_2, x_3, x_4 & \leq 1, \\
x_1, x_2, x_3, x_4 & \geq 0.
\end{align*}

And in standard form...
$$
\min_{x\in\{0,1\}^4} -16 x_1 - 22 x_2 - 12 x_3 - 8 x_4,
$$
subject to
\begin{align*}
5 x_1 + 7 x_2 + 4 x_3 + 3 x_4 + e_1 &= 14, \\
x_1 + e_2  &= 1, \\
x_2 + e_3  &= 1, \\
x_3 + e_4  &= 1, \\
x_4 + e_5  &= 1, \\
x_1, x_2, x_3, x_4, e_1, e_2, e_3, e_4, e_5 & \geq 0.
\end{align*}

In [ ]:
matrix_a = np.array(
    [
        [5, 7, 4, 3, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 1],
    ]
)


In [ ]:
vector_b = np.array([14, 1, 1, 1, 1])


In [ ]:
vector_c = np.array([-16, -22, -12, -8, 0, 0, 0, 0, 0])


We create the algorithm

In [ ]:
the_algorithm = SimplexAlgorithmTableau(
    objective=vector_c,
    constraint_matrix=matrix_a,
    right_hand_side=vector_b,
)


We solve the problem

In [ ]:
optimal_tableau: SimplexTableau = the_algorithm.solve()


Check if the problem is feasible

In [ ]:
if optimal_tableau is None:
    warnings.warn(
        f'Optimization problem is infeasible. There must be a problem in the formulation.'
    )
    sys.exit()


Optimal solution

In [ ]:
print(optimal_tableau.feasible_basic_solution)



Optimal value

In [ ]:
print(f'{optimal_tableau.value_objective_function:.3g}')


Decision variables.

Decision for China

In [ ]:
x_china = optimal_tableau.feasible_basic_solution[0]
print(f'Decision for China: {x_china:.3g}')


Decision for Iran

In [ ]:
x_iran = optimal_tableau.feasible_basic_solution[1]
print(f'Decision for Iran: {x_iran:.3g}')


Decision for Brazil

In [ ]:
x_brazil = optimal_tableau.feasible_basic_solution[2]
print(f'Decision for Brazil: {x_brazil:.3g}')


Decision for India

In [ ]:
x_india = optimal_tableau.feasible_basic_solution[3]
print(f'Decision for India: {x_india:.3g}')


For the company, this suggests to build dams 1 and 2, and not to build dam
4. Regarding dam 3, building it would not fit the budget. Building
dams 1 and 2 will cost 120 mCHF, and the expected benefit is 380 mCHF.

## Question 4
Solve the problem by full enumeration, and use the
solution  to advise the company. Compare the two
advises.

We code a function that calculates the revenues.

In [ ]:
def revenues(x_1: int, x_2: int, x_3: int, x_4: int) -> float:
    """Calculates the objective function

    :param x_1: decision for China
    :param x_2: decision for Iran
    :param x_3: decision for Brazil
    :param x_4: decision for India
    :return: total revenues
    """
    return 16 * x_1 + 22 * x_2 + 12 * x_3 + 8 * x_4



We code a function that calculates the budget.

In [ ]:
def budget(x_1: int, x_2: int, x_3: int, x_4: int) -> float:
    """Calculates the budget

    :param x_1: decision for China
    :param x_2: decision for Iran
    :param x_3: decision for Brazil
    :param x_4: decision for India
    :return: budget
    """
    return 5 * x_1 + 7 * x_2 + 4 * x_3 + 3 * x_4



We code a function that verifies the budget constraint

In [ ]:
def budget_constraint(x_1: int, x_2: int, x_3: int, x_4: int) -> bool:
    """Check the budget constraint

    :param x_1: decision for China
    :param x_2: decision for Iran
    :param x_3: decision for Brazil
    :param x_4: decision for India
    :return: True if the budget constraint is verified, False otherwise.
    """
    the_budget = budget(x_1, x_2, x_3, x_4)
    return the_budget <= 14



Now, we perform a complete enumeration to identify the best solution.

In [ ]:
best_solution = None
best_value = 0


Loop over all possible combinations of decisions to identify the best one.

In [ ]:
for x_1, x_2, x_3, x_4 in itertools.product([0, 1], repeat=4):
    if budget_constraint(x_1, x_2, x_3, x_4):
        the_revenues = revenues(x_1, x_2, x_3, x_4)

        if the_revenues > best_value:
            best_solution = x_1, x_2, x_3, x_4
            best_value = the_revenues


Optimal solution

In [ ]:
print(best_solution)


Optimal value

In [ ]:
total_revenues = -optimal_tableau.value_objective_function
print(f'Total revenues: {total_revenues:.3g}')


Budget

In [ ]:
total_budget = budget(*best_solution)
print(f'Total budget: {total_budget:.3g}')


Check that the budget constraint is verified

In [ ]:
is_budget_constraint_verified = budget_constraint(*best_solution)
if is_budget_constraint_verified:
    print('Budget constraint is verified')
else:
    warnings.warn('Budget constraint is violated.')


The optimal solution is to build dams in Iran ($x_2=1$), Brazil ($x_3=1$) and India ($x_4=1$). The budget is
completely invested, and the expected revenues are 420
mCHF. Although the dam in China has the highest return, it consumes too
much of the budget. It is therefore better not to build it, and invest
in the three others. It illustrates the fact that using the relaxed problem
to make the decision yields to a sub-optimal
solution.